<a href="https://colab.research.google.com/github/sanhiitaa/100-days-of-deep-learning/blob/main/deep_learning_cookbook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense

* `n1, n2, n3... `- has to be set based on requirement.
* `x` - total input features


### **Early Stopping**

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
# defining the model
model= Sequential()

model.add(Dense(n1, activation='relu', input_dim=2))
model.add(Dense(n2, activation='sigmoid'))

In [ ]:
# compiling the model
model.compile(loss=..., optimizer=..., metrics=['...'])

In [ ]:
# setting up EarlyStopping callback
callback= EarlyStopping(
    monitor='val_loss', # quantity to be monitored
    min_delta=0.00001,  # change in the quantity to qualify as improvement
    patience=20,        # number of epochs to wait before stopping the training
    verbose=1,          # callback message display switch
    mode='auto',        # auto/min/max - monitors if the quantity should increase/decrease
    baseline=None,      # training stops if no improvement over baseline
    restore_best_weights=True
)

In [ ]:
# fitting the model with the callback
history= model.fit(X_train, y_train, validation_data=(X_test, y_test), callbacks=callback, epochs=...)

### **Dropout layers**

In [ ]:
from tensorflow.keras.layers import Dropout

In [ ]:
model= Sequential()

model.add(Dense(n1, activation='relu', input_dim=x))
model.add(Dropout(0.5))
model.add(Dense(n2, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(n3, activation='sigmoid'))

In `model.add(Dropout(0.5))`, 0.5 means 50% of the nodes will be dropped, this number can be adjusted accordingly.

### **Regularization**

In [ ]:
model= Sequential()

model.add(Dense(n1, activation='relu', input_dim=x, kernel_regularizer=tensorflow.keras.regularizers.l1(0.001)))
model.add(Dense(n2, activation='relu', kernel_regularizer= tensorflow.keras.regularizers.l2(0.001)))
model.add(Dense(n3, activation='sigmoid'))

model.summary()

### **Xavier (Gloarat) and He Weight Initialization**

In [ ]:
model=Sequential()

model.add(Dense(n1, activation='relu', input_dim=x, kernel_initializer='he_normal'))

`kernel_initializer` can be initialized with:

1. `glorat_uniform`; `glorat_normal` ▶ `tanh`; `sigmoid`
2. `he_uniform`; `he_normal` ▶ `relu`

### **Batch Normalization**

In [ ]:
from tensorflow.keras.layers import BatchNormalization

In [ ]:
model=Sequential()

model.add(Dense(n1, activation='relu', input_dim=x))
model.add(BatchNormalization())
model.add(Dense(n2, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(n3, activation='sigmoid'))

model.summary()

### **SGD; Momentum; NAG (Nesterov Accelerated Gradient)**

In [ ]:
from tensorflow.keras.optimizers import SGD

In [ ]:
# SGD
optimizer=SGD(learning_rate=0.01, momentum=0, nesterov=False, name='SGD')

# Momentum
optimizer=SGD(learning_rate=0.01, momentum=beta, nesterov=False, name='SGD')

# NAG  implementation
optimizer=SGD(learning_rate=0.01, momentum=beta, nesterov=True, name='SGD')

`beta` is `β - decaying factor`.
Usually set at `β=0.9`.

In [ ]:
model.compile(optimizer=optimizer, loss='...', metrics=['...'])

### **Keras Tuner**
Tunes all parameters mentioned below:

1. Number of layers
2. Number of nodes
3. Activation function
4. Optimizer
5. Dropout layers



In [ ]:
# installing keras-tuner
!pip install -q keras-tuner

In [ ]:
# importing keras_tuner
import keras_tuner as kt

In [ ]:
# defining a model builder
def model_builder(hp):

  # initializing sequential class
  model= Sequential()

  # setting a counter to make the initial layer
  counter=0

  # initializing the for loop to create num_layers
  for i in range(hp.Int('num_layers', min_value=1, max_value=10)):
    if counter==0:
      # adding the first layer if counter=0
      model.add(
          Dense(
              units=hp.Int('units-'+str(i), min_value=8, max_value=128, step=8), # number of neurons
              activation=hp.Choice('activation-'+str(i), values=['relu', 'tanh', 'sigmoid']), # activation layer
              input_dim=8
              )
          )
      model.add(Dropout(hp.Choice('dropout-'+str(i), values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
      counter+=1
    else:
      # adding subsequent layers if counter!=0
      model.add(
          Dense(
              units=hp.Int('units-'+str(i), min_value=8, max_value=128, step=8), # number of neurons
              activation=hp.Choice('activation-'+str(i), values=['relu', 'tanh', 'sigmoid']) # activation layer
              )
          )
      model.add(Dropout(hp.Choice('dropout-'+str(i), values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]))) # dropout layer
  model.add(Dense(1, activation='sigmoid')) # output layer

  # compiling the model
  model.compile(optimizer=hp.Choice('optimizer', values=['adam', 'rmsprop','sgd','adadelta','nadam']),
                loss='binary_crossentropy',
                metrics=['accuracy'])
  return model

In [ ]:
# initializing a tuner of kerastuner
tuner= kt.RandomSearch(model_builder,
                       objective='val_accuracy',
                       max_trials=10,
                       directory='mydir',
                       project_name='tuner-with-dropout')

In [ ]:
# searching for optimum parameters
tuner.search(x_train, y_train, epochs=5, validation_data=(x_test, y_test))

In [ ]:
# to get the most optimum hyperparameters
tuner.get_best_hyperparameters()[0].values

In [ ]:
# extracting the best model
best_model=tuner.get_best_models()[0]

In [ ]:
# model summary
best_model.summary()

In [ ]:
# completing best model training
best_model.fit(x_train, y_train, epochs=100, initial_epoch=6, validation_data=(x_test, y_test))